In [1]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 
from numpy.polynomial.polynomial import Polynomial

from keras.models import Sequential
from keras.layers import Dense

from progressbar import ProgressBar

from sklearn.preprocessing import StandardScaler

In [3]:

#import training datadet
inter = pd.ExcelFile('training data_Github.xlsx')
inter = pd.read_excel(inter,'Combined')

comb = inter.dropna(axis=0)
comb = comb[comb["tR (min)"] != "?"]
col_to_remove=[]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
        col_to_remove.append(col)
comb = comb.reset_index(drop=True)
X_CAL = comb.drop(['Project','Column','Compound','tR (min)'], axis=1)
print(X_CAL.info())
y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
y = pd.DataFrame(comb, columns = ['tR (min)'])
print(y.info())
#import test data
inter1 = pd.ExcelFile('test data_Github.xlsx')
inter1 = pd.read_excel(inter1,'Sheet2')

test = inter1.dropna(axis=0)
test = test[test["tR (min)"] != "?"]
#preprocess
test = test.drop(col_to_remove,axis=1)
X_test = test.drop(['Project','Column','Compound','tR (min)'], axis=1)
print(X_test.info())
y_test = {'tR (min)': [round(x,4) for x in test['tR (min)']]}
y_test = pd.DataFrame(test, columns = ['tR (min)'])
print(y_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Columns: 141 entries, H to fsulfone
dtypes: float64(100), int64(41)
memory usage: 1.9 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tR (min)  1790 non-null   float64
dtypes: float64(1)
memory usage: 14.1 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 143
Columns: 141 entries, H to fsulfone
dtypes: float64(100), int64(41)
memory usage: 110.9 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 143
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tR (min)  100 non-null    float64
dtypes: float64(1)
memory usage: 1.6 KB
None


In [9]:

def HPLC_COLUMN_TRANSFER_independent_test(X_CAL,Y_CAL,X_test,Y_test,Project_IN):
    
    UNIQUE_LIST = test['Column'].unique()
    print(UNIQUE_LIST)
    print(test['Project'].unique())
    RMSEP_1 = []
    RMSEP_2 = []
    LABEL = []
    Project = Project_IN  # Example project

    for i in range(5):
        for k in range(i + 1, 5):
            ind_cal = (test['Project'] == Project) & (test['Column'] == UNIQUE_LIST[i])
            ind_val = (test['Project'] == Project) & (test['Column'] == UNIQUE_LIST[k])

            if ind_cal.sum() < 3 or ind_val.sum() < 3:
                continue
            else: 

                # ANN
                XScaler=StandardScaler()
                YScaler=StandardScaler()
                temp_x=np.vstack([X_CAL, X_test.loc[ind_cal,:]])
                temp_y=np.vstack([y, Y_test.loc[ind_cal,:]])
                temp_x_test = X_test.loc[ind_val,:]
                temp_y = np.asarray(temp_y).astype('float32')
                temp_x = np.asarray(temp_x).astype('float32')
                temp_x_test = np.asarray(temp_x_test).astype('float32')
    
                X_scaled=XScaler.fit_transform(temp_x)
                y_scaled=YScaler.fit_transform(temp_y)
                X_test_scaled=XScaler.transform(temp_x_test)

                # create ANN model
                model = Sequential()
                # Defining the Input layer and FIRST hidden layer, both are same!
                model.add(Dense(units=5, input_dim=141, kernel_initializer='normal', activation='relu'))
                # Defining the Second layer of the model
                # after the first layer we don't have to specify input_dim as keras configure it automatically
                model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
                # Defining the Third layer of the model
                model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
                # The output neuron is a single fully connected node 
                # Since we will be predicting a single number
                model.add(Dense(1, kernel_initializer='normal'))
                # Compiling the model
                model.compile(loss='mean_squared_error', optimizer='adam')    
                # Fitting the ANN to the Training set
                model.fit(X_scaled, y_scaled ,batch_size = 50, epochs = 50, verbose=0)

                
                # Generating Predictions on testing data
                Predictions=model.predict(X_test_scaled)
                # Scaling the predicted Price data back to original price scale
                Predictions=YScaler.inverse_transform(Predictions)
                RMSEP_temp = np.sqrt(mean_squared_error(Y_test.loc[ind_val], Predictions))
                tempA=test.loc[ind_cal, 'Compound'].values
                tempA = [[item] for item in tempA]
                tempB=test.loc[ind_val, 'Compound'].values
                tempB = [[item] for item in tempB]
                
                LABEL.append([
                    UNIQUE_LIST[i],
                    tempA,
                    UNIQUE_LIST[k],
                    tempB,
                    ind_val.sum(),
                    Y_test.loc[ind_val], 
                    Predictions
                ])

                RMSEP_2.append(RMSEP_temp)

               

    OUTPUT = {
        'LABEL': LABEL,
        'RMSEP_2': RMSEP_2
    }
    
    return OUTPUT

In [16]:
#OUTPUT=HPLC_COLUMN_TRANSFER_independent_test(X_CAL,y,X_test,y_test,'Project AA')
#OUTPUT=HPLC_COLUMN_TRANSFER_independent_test(X_CAL,y,X_test,y_test,'Project BB')
#OUTPUT=HPLC_COLUMN_TRANSFER_independent_test(X_CAL,y,X_test,y_test,'Project CC')
OUTPUT=HPLC_COLUMN_TRANSFER_independent_test(X_CAL,y,X_test,y_test,'Project DD')

['Zorbax Extend C18' 'Kinetix XB-C18' 'Poroshell EC-C18' 'Cortecs C18+'
 'ACE C18']
['Project CC' 'Project AA' 'Project BB' 'Project DD']
1/1 [==============================] - 0s 62ms/step


In [17]:
#print(OUTPUT['LABEL'][0][1])

samp=[]
columnA=[]
columnB=[]
CompoundA=np.array
CompoundB=np.array
Rt_mea=np.array
Rt_pre=np.array
rmse_ANN=[]

for i in range(len(OUTPUT['LABEL'])):
    columnA.append(OUTPUT['LABEL'][i][0])
    columnB.append(OUTPUT['LABEL'][i][2])
    tempA=np.array(OUTPUT['LABEL'][i][1])
    CompoundA=np.vstack([CompoundA,tempA])
    tempB=np.array(OUTPUT['LABEL'][i][3])
    CompoundB=np.vstack([CompoundB,tempB])
    samp.append(OUTPUT['LABEL'][i][4])
    temp_mea=np.array(OUTPUT['LABEL'][i][5])
    Rt_mea=np.vstack([Rt_mea,temp_mea])
    temp_pre=np.array(OUTPUT['LABEL'][i][6])
    Rt_pre=np.vstack([Rt_pre,temp_pre])
    rmse_ANN.append(OUTPUT['RMSEP_2'][i])


tempA=np.array(CompoundA)
CompoundA=tempA.reshape(-1,1)

tempB=np.array(CompoundB)
CompoundB=tempB.reshape(-1,1)

columnA=pd.DataFrame(columnA,columns=['columnA'])
columnB=pd.DataFrame(columnB,columns=['columnB'])
CompoundA=pd.DataFrame(CompoundA[1:,:],columns=['CompoundA'])
CompoundB=pd.DataFrame(CompoundB[1:,:],columns=['CompoundB'])
samp=pd.DataFrame(samp,columns=['# of samples'])
Rt_mea=pd.DataFrame(Rt_mea[1:,:],columns=['measured Rt'])
Rt_pre=pd.DataFrame(Rt_pre[1:,:],columns=['predicted Rt'])
rmse_ANN=pd.DataFrame(rmse_ANN,columns=['rmse_ANN'])

results = pd.concat([columnA, columnB, CompoundA, CompoundB, samp, Rt_mea, Rt_pre, rmse_ANN],axis=1)

#results.to_csv('COLUMN_TRANSFER_Project AA.csv')
#results.to_csv('COLUMN_TRANSFER_Project BB.csv')
#results.to_csv('COLUMN_TRANSFER_Project CC.csv')
results.to_csv('COLUMN_TRANSFER_Project DD.csv')